In [1]:
import os
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import pickle
import re
import os
import sys
from dataclasses import dataclass, field
from typing import Optional
import datasets
from datasets import ClassLabel, load_dataset, Dataset, DatasetDict
import string
from typing import Dict ,List
import transformers

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from transformers import BertTokenizerFast
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
DATA_PATH = "/Users/gufran/Developer/Projects/AI/MawqifStanceDetection/data"
MODEL_PATH = "/Users/gufran/Developer/Projects/AI/MawqifStanceDetection/models"

model_name = "aubmindlab/bert-base-arabertv02-twitter"
# model_name = "aubmindlab/bert-base-arabertv02"

In [3]:
df = pd.read_csv(os.path.join(DATA_PATH, "cleaned.csv"))

mapping_sent = {'Neutral': 2, 'Positive': 1, 'Negative': 0}
df['sentiment'] = df['sentiment'].apply(lambda x: mapping_sent[x])

mapping_sarcasm = {"Yes": 1, "No": 0}
df['sarcasm'] = df['sarcasm'].apply(lambda x: mapping_sarcasm[x])

In [4]:
X = df[["text"]]
y = df[["sentiment", "sarcasm"]]

In [5]:
arabic_punctuations = '''`÷×؛<>()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_hash_URL_MEN(text):
    text = re.sub(r'#',' ',text)
    text = re.sub(r'_',' ',text)
    text = re.sub(r'URL','',text)
    text = re.sub(r'MENTION','',text)
    return text

def normalize_arabic(text):
    text = re.sub("[إآ]", "ا", text)
    #text = re.sub("ى", "ي", text)
    #text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def process_tweet(tweet):     
    tweet=remove_hash_URL_MEN(tweet)
    tweet = re.sub('@[^\s]+', ' ', str(tweet))
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(tweet))    
    tweet= normalize_arabic(str(tweet))
    
    return tweet

arabert_prep = ArabertPreprocessor(model_name=model_name)
X.text = X.text.apply(lambda x: process_tweet(x))
X.text = X.text.apply(lambda x: arabert_prep.preprocess(x))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [7]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2976, 1), (2976, 2), (526, 1), (526, 2))

In [8]:
def encode_text(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

tokenizer = AutoTokenizer.from_pretrained(model_name)
encoded_texts_train = [encode_text(text) for text in X_train["text"]]
encoded_texts_test = [encode_text(text) for text in X_test["text"]]

In [9]:
encoded_texts_train[0].attention_mask.shape, encoded_texts_train[1].attention_mask.shape

(torch.Size([1, 128]), torch.Size([1, 128]))

In [10]:
num_sarcasm_labels = len(y_train.sarcasm.unique())
num_sentiment_labels = len(y_train.sentiment.unique())

In [24]:
class MultiTaskModel(nn.Module):
    def __init__(self, base_model):
        super(MultiTaskModel, self).__init__()
        self.base_model = base_model
        self.hidden_layer = nn.Linear(base_model.config.hidden_size, 512)
        self.dropout = nn.Dropout(0.2)
        self.sarcasm_classifier = nn.Linear(512, num_sarcasm_labels)
        self.sentiment_classifier = nn.Linear(512, num_sentiment_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        hidden_output = self.dropout(F.relu(self.hidden_layer(last_hidden_state)))
        sarcasm_logits = self.sarcasm_classifier(hidden_output)
        sentiment_logits = self.sentiment_classifier(hidden_output)
        return sarcasm_logits, sentiment_logits

In [22]:
dataset_train = TensorDataset(
    torch.cat([item["input_ids"] for item in encoded_texts_train]),
    torch.cat([item["attention_mask"] for item in encoded_texts_train]),
    torch.tensor(y_train["sarcasm"].values),
    torch.tensor(y_train["sentiment"].values)
)
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)

dataset_val = TensorDataset(
    torch.cat([item["input_ids"] for item in encoded_texts_test]),
    torch.cat([item["attention_mask"] for item in encoded_texts_test]),
    torch.tensor(y_test["sarcasm"].values),
    torch.tensor(y_test["sentiment"].values))
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=True)

In [ ]:
w1 = 0.9
w2 = 0.1
while w1 > 0.1:
    device = torch.device("mps")

    encoder_model = AutoModel.from_pretrained(model_name)
    multi_task_model = MultiTaskModel(encoder_model)

    multi_task_model.to(device)
    optimizer = AdamW(multi_task_model.parameters(), lr=1e-5)
    num_epochs = 10
    
    print("----------------------------------------------------------")
    print(f"Training with w1:{w1} and w2:{w2}\n")

    for epoch in range(num_epochs):
        multi_task_model.train()
        total_loss = 0
        correct_sarcasm = 0
        correct_sentiment = 0
        total_samples = 0

        for batch in tqdm(train_loader, desc="Training"):
            input_ids, attention_mask, sarcasm_labels, sentiment_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            sarcasm_labels = sarcasm_labels.to(device)
            sentiment_labels = sentiment_labels.to(device)
            
            optimizer.zero_grad()
            sarcasm_logits, sentiment_logits = multi_task_model(input_ids, attention_mask)
            sarcasm_loss = nn.CrossEntropyLoss()(sarcasm_logits, sarcasm_labels)
            sentiment_loss = nn.CrossEntropyLoss()(sentiment_logits, sentiment_labels)
            loss = w1*sarcasm_loss + w2*sentiment_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, sarcasm_predicted = torch.max(sarcasm_logits, 1)
            _, sentiment_predicted = torch.max(sentiment_logits, 1)
            correct_sarcasm += (sarcasm_predicted == sarcasm_labels).sum().item()
            correct_sentiment += (sentiment_predicted == sentiment_labels).sum().item()
            total_samples += input_ids.size(0)

In [13]:
w1 = 0.9
w2 = 0.1
while w1 > 0.1:
    device = torch.device("mps")

    encoder_model = AutoModel.from_pretrained(model_name)
    multi_task_model = MultiTaskModel(encoder_model)

    multi_task_model.to(device)
    optimizer = AdamW(multi_task_model.parameters(), lr=1e-5)
    num_epochs = 10
    
    print("----------------------------------------------------------")
    print(f"Training with w1:{w1} and w2:{w2}\n")

    for epoch in range(num_epochs):
        multi_task_model.train()
        total_loss = 0
        correct_sarcasm = 0
        correct_sentiment = 0
        total_samples = 0

        for batch in tqdm(train_loader, desc="Training"):
            input_ids, attention_mask, sarcasm_labels, sentiment_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            sarcasm_labels = sarcasm_labels.to(device)
            sentiment_labels = sentiment_labels.to(device)
            
            print(input_ids)

            optimizer.zero_grad()
            sarcasm_logits, sentiment_logits = multi_task_model(input_ids, attention_mask)
            sarcasm_loss = nn.CrossEntropyLoss()(sarcasm_logits, sarcasm_labels)
            sentiment_loss = nn.CrossEntropyLoss()(sentiment_logits, sentiment_labels)
            loss = w1*sarcasm_loss + w2*sentiment_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, sarcasm_predicted = torch.max(sarcasm_logits, 1)
            _, sentiment_predicted = torch.max(sentiment_logits, 1)
            correct_sarcasm += (sarcasm_predicted == sarcasm_labels).sum().item()
            correct_sentiment += (sentiment_predicted == sentiment_labels).sum().item()
            total_samples += input_ids.size(0)

        avg_loss = total_loss / len(train_loader)
        sarcasm_accuracy = correct_sarcasm / total_samples
        sentiment_accuracy = correct_sentiment / total_samples

        multi_task_model.eval()
        with torch.no_grad():
            total_val_loss = 0
            val_correct_sarcasm = 0
            val_correct_sentiment = 0
            total_val_samples = 0

            for val_batch in tqdm(val_loader, desc="Validation"):
                val_input_ids, val_attention_mask, val_sarcasm_labels, val_sentiment_labels = val_batch
                val_input_ids = val_input_ids.to(device)
                val_attention_mask = val_attention_mask.to(device)
                val_sarcasm_labels = val_sarcasm_labels.to(device)
                val_sentiment_labels = val_sentiment_labels.to(device)

                val_sarcasm_logits, val_sentiment_logits = multi_task_model(val_input_ids, val_attention_mask)
                val_sarcasm_loss = nn.CrossEntropyLoss()(val_sarcasm_logits, val_sarcasm_labels)
                val_sentiment_loss = nn.CrossEntropyLoss()(val_sentiment_logits, val_sentiment_labels)
                val_loss = w1*val_sarcasm_loss + w1*val_sentiment_loss

                total_val_loss += val_loss.item()
                _, val_sarcasm_predicted = torch.max(val_sarcasm_logits, 1)
                _, val_sentiment_predicted = torch.max(val_sentiment_logits, 1)
                val_correct_sarcasm += (val_sarcasm_predicted == val_sarcasm_labels).sum().item()
                val_correct_sentiment += (val_sentiment_predicted == val_sentiment_labels).sum().item()
                total_val_samples += val_input_ids.size(0)

            avg_val_loss = total_val_loss / len(val_loader)
            val_sarcasm_accuracy = val_correct_sarcasm / total_val_samples
            val_sentiment_accuracy = val_correct_sentiment / total_val_samples

        print(f"Epoch {epoch+1},\n Training - Avg Loss: {avg_loss:.4f}, Sarcasm Acc: {sarcasm_accuracy:.4f}, Sentiment Acc: {sentiment_accuracy:.4f},\n Validation - Avg Loss: {avg_val_loss:.4f}, Sarcasm Val Acc: {val_sarcasm_accuracy:.4f}, Sentiment Val Acc: {val_sentiment_accuracy:.4f}", end="\n\n")
    
    formatted_w1 = f'{w1:.1f}'
    formatted_w2 = f'{w2:.1f}'
    torch.save(multi_task_model.base_model.state_dict(), os.path.join(MODEL_PATH, f'BERT_sar_sent_{formatted_w1}_{formatted_w2}.pth'))
    w1-=0.1
    w2+=0.1

Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


----------------------------------------------------------
Training with w1:0.9 and w2:0.1



Validation: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 12.96it/s]


Epoch 1,
 Training - Avg Loss: 0.2645, Sarcasm Acc: 0.9496, Sentiment Acc: 0.4933,
 Validation - Avg Loss: 0.9296, Sarcasm Val Acc: 0.9468, Sentiment Val Acc: 0.6350



Validation: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 13.87it/s]


...

Epoch 8,
 Training - Avg Loss: 0.0774, Sarcasm Acc: 0.9543, Sentiment Acc: 0.9835,
 Validation - Avg Loss: 0.1341, Sarcasm Val Acc: 0.9468, Sentiment Val Acc: 0.6787



Validation: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 12.42it/s]


Epoch 9,
 Training - Avg Loss: 0.0611, Sarcasm Acc: 0.9546, Sentiment Acc: 0.9859,
 Validation - Avg Loss: 0.1494, Sarcasm Val Acc: 0.9468, Sentiment Val Acc: 0.6825



Validation: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 12.44it/s]


Epoch 10,
 Training - Avg Loss: 0.0476, Sarcasm Acc: 0.9560, Sentiment Acc: 0.9899,
 Validation - Avg Loss: 0.1511, Sarcasm Val Acc: 0.9468, Sentiment Val Acc: 0.6806

